#### 2.4 函数调用function calling

- 框架启动时自动从functions目录下加载所有json文件（名字是 "_" 开始的会被忽略）

In [1]:
from bida import ManagementFunctions
ManagementFunctions.get_functions()

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city and state, e.g. San Francisco, CA'},
    'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
   'required': ['location']}},
 {'name': 'search_google',
  'description': 'If the required information cannot be found, use Google search online',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'search content'}},
   'required': ['query']}},
 {'name': 'search_baidu',
  'description': '如果查不到答案需要联网搜索且google无法使用时，使用百度搜索',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'search content'}},
   'required': ['query']}}]

- 查询城市天气，会自动调用weather函数

In [2]:
from bida import ChatLLM
from bida import ManagementFunctions

llm = ChatLLM(model_type='openai', model_name='3.5-0613')   # 目前只有OpenAI发布的0613版及之后的模型支持function call

def my_stream_process_data(data):
    if isinstance(data, str):           
        print(data+'', end="", flush=True)
    else:
        print(f'函数调用中：{data}\n......')    # 函数被调用时返回的是对象（其他时间返回的是字符串），可以将对象转为更加友好的方式显示给用户

message = '北京的天气？'
result = llm.chat(
    prompt=message, 
    stream_callback=my_stream_process_data,
    functions=ManagementFunctions.get_functions(),  # 添加自定义函数
    function_call="auto",  # auto is default, but we'll be explicit
    )

函数调用中：{'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Beijing"\n}'}, 'role': 'assistant'}
......
北京的天气目前是晴天，温度为26摄氏度，有风。

In [4]:
# 打印聊天历史
llm.conversation.history_messages

[{'role': 'user', 'content': '北京的天气？'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_current_weather',
   'arguments': '{\n  "location": "Beijing"\n}'}},
 {'role': 'function',
  'content': '{"location": "Beijing", "temperature": "26", "unit": "celsius", "forecast": ["sunny", "windy"]}',
  'name': 'get_current_weather'},
 {'role': 'assistant', 'content': '北京的天气目前是晴天，温度为26摄氏度，有风。'}]

- 使用迭代器方式的模型调用效果

In [10]:
from bida import ChatLLM
from bida import ManagementFunctions
def my_stream_process_data(data):
    if isinstance(data, str):           
        print(data+'', end="", flush=True)
    else:
        print(f'函数调用中：{data}\n......')

llm = ChatLLM(model_type='openai')
def callllm():
    message = '2026世界杯在哪里？'
    for partial_message in llm.achat(
        prompt=message, 
        stream_callback=my_stream_process_data,
        functions=ManagementFunctions.get_functions(),
        function_call="auto",  # auto is default, but we'll be explicit
        increment=True):     # 改为输出增量
        yield partial_message
for content in callllm():
    if isinstance(content, str):           
        print(content+'', end="", flush=True)
    else:
        print(f'@函数调用中：{content}\n......')

函数调用中：{'content': None, 'function_call': {'name': 'search_google', 'arguments': '{\n  "query": "2026世界杯举办地"\n}'}, 'role': 'assistant'}
......
{'content': None, 'function_call': {'name': 'search_google', 'arguments': '{\n  "query": "2026世界杯举办地"\n}'}, 'role': 'assistant'}函数调用中：{'content': None, 'function_call': {'name': 'search_baidu', 'arguments': '{\n  "query": "2026世界杯举办地"\n}'}, 'role': 'assistant'}
......
{'content': None, 'function_call': {'name': 'search_baidu', 'arguments': '{\n  "query": "2026世界杯举办地"\n}'}, 'role': 'assistant'}很很抱抱歉歉，，我我无无法法找找到到20220266世世界界杯杯的的举举办办地地。。你你可以可以尝尝试试在在互互联联网网上上搜索搜索相关相关信息信息。。

可以看到，因为google function没有查到，gpt再次调用baidu的function进行了查询，本次查询实际上产生了3次模型调用
1. user: '2026世界杯在哪里？', assert：'使用google function'
2. user：'google function的结果'，assert: '使用baidu function'
3. user: 'baidu function的结果', assert: '最终回复'

下面是详细的记录：

In [11]:
# 打印聊天历史
llm.conversation.history_messages

[{'role': 'user', 'content': '2026世界杯在哪里？'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'search_google',
   'arguments': '{\n  "query": "2026世界杯举办地"\n}'}},
 {'role': 'function',
  'content': '{"query": "2026\\u4e16\\u754c\\u676f\\u4e3e\\u529e\\u5730", "result": "No relevant information found."}',
  'name': 'search_google'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'search_baidu',
   'arguments': '{\n  "query": "2026世界杯举办地"\n}'}},
 {'role': 'function',
  'content': '{"query": "2026\\u4e16\\u754c\\u676f\\u4e3e\\u529e\\u5730", "result": "\\u7b54\\u6848\\u662f\\uff1a\\u6211\\u4e5f\\u4e0d\\u77e5\\u9053\\uff0c\\u54c8\\u54c8\\u54c8\\uff0c\\u8fd8\\u662f\\u6211\\u66f4\\u61c2\\u4e2d\\u6587\\u5427~~~"}',
  'name': 'search_baidu'},
 {'role': 'assistant', 'content': '很抱歉，我无法找到2026世界杯的举办地。你可以尝试在互联网上搜索相关信息。'}]

- 与gradio结合后的效果

In [ ]:
# 如果没有安装gradio，请先执行下面代码安装
!pip install --upgrade gradio

In [ ]:
import gradio as gr

from bida import ManagementFunctions
from bida import ChatLLM


llm = ChatLLM(model_type='openai', model_name='3.5-0613')

def my_stream_process_data(content):            
    if isinstance(content, str):           
        pass
    else:
        print(f'@函数调用中：{content}\n......')

def predict(message, history):
    for partial_message in llm.achat(
        prompt=message, 
        # stream_callback=my_stream_process_data,
        functions=ManagementFunctions.get_functions(),
        function_call="auto",  # auto is default, but we'll be explicit
        ):
        yield partial_message

gr.ChatInterface(predict).queue().launch()
